In [26]:
from keras.datasets import mnist
import numpy as np
from numpy import array
from math import log
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from itertools import product
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import MiniBatchSparsePCA
from sklearn.svm import SVC
from sklearn.feature_selection import *
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score
import tensorflow as tf
import copy
from skimage.transform import resize
from skimage.util import *

#load MNIST
(X_train_raw, y_train_raw), (X_test_raw, y_test_raw) = mnist.load_data()

Using TensorFlow backend.


In [27]:
def image_set_preprocessing(X, y, batch_ratio = 1, pad_size=2, pad_method='constant'):
    X_pad = np.pad(X, ((0,0),(pad_size,pad_size),(pad_size,pad_size)), 'constant')
    
    batch_size = (int)(X_pad.shape[0]*batch_ratio)
    
    order = np.array(range(X_pad.shape[0]))
    np.random.shuffle(order)
    X_pad_shuffle = X_pad[order]
    y_shuffle = y[order]

    X_train_batch = ((X_pad_shuffle[0:batch_size, :, :]).astype('float32'))/255
    y_train_batch = y_shuffle[0:batch_size,]
    X_train_batch = X_train_batch.reshape(X_train_batch.shape[0], X_train_batch.shape[1], X_train_batch.shape[2], 1)
    
    return X_train_batch, y_train_batch

def flatten_patches(cubio, size):
    window_shape = (1,size,size,cubio.shape[3])
    step = (1,size,size,cubio.shape[3])
    patches = view_as_windows(cubio, window_shape, step)
    patches = patches.squeeze(axis = (3,4))
    patches_panel = patches.reshape(-1, patches.shape[-3]*patches.shape[-2]*patches.shape[-1])
    
    return patches_panel

def flatten_patches_v2(cubio):
    window_shape = (1,2,2,cubio.shape[3])
    step = (1,2,2,cubio.shape[3])
    patches = view_as_windows(cubio, window_shape, step)
    patches = patches.squeeze(axis = (3,4))
    patches_panel = patches.reshape(-1, patches.shape[-3]*patches.shape[-2]*patches.shape[-1])
    
    return patches_panel

def remove_low_variance(patches_panel,thr=0.05):
    std = np.std(patches_panel, axis = 1)
    patches_clean = patches_panel[std>thr]
    
    return patches_clean

def remove_patches_mean(patches):
    mean = patches.mean(axis = 1)
    patches_mean_remov = (patches.T-mean).T
    
    return patches_mean_remov

def pca_kernel(patches, n_comps, kernel):
    pca = PCA(n_components = n_comps)
    pca.fit(patches)
    kernel.append(pca)  
    
def pca_transform(kernel, layer, patches_mean_remov, patches_panel, X_train):
    n_sample = X_train.shape[0]
    h = (int)(X_train.shape[1]/pow(2,layer))
    patches_proj = k[layer-1].transform(patches_mean_remov)
    cubio_pos = patches_proj.reshape(n_sample,h,h,-1)
    cubio_neg = -cubio_pos
    cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=3)
    dc = patches_panel.mean(axis=1)*2
    dc = dc.reshape(n_sample, h, h, -1)
    cubio_next = np.concatenate((cubio_pca, dc), axis=3)
    
    return cubio_next

def dc_add(X_train, layer, cubio, cubio_pca):
    n_sample = 60000
    cubio_enlarge = np.zeros((n_sample,32*pow(2,1),32*pow(2,1),cubio.shape[3]))
    for i in range(n_sample):
        cubio_enlarge[i,:,:,:] = resize(cubio[i,:,:,:], output_shape=[cubio.shape[1]*2,cubio.shape[2]*2,cubio.shape[3]])
    window_shape = (1,2,2,cubio_enlarge.shape[3])
    step = (1,2,2,cubio_enlarge.shape[3])
    patches = view_as_windows(cubio_enlarge, window_shape, step)
    patches = patches.squeeze(axis = (3,4))
    patches_panel = patches.reshape(-1, patches.shape[-3]*patches.shape[-2]*patches.shape[-1])

    dc = patches_panel.mean(axis=1)*2
    dc = dc.reshape(n_sample, 32, 32, -1)
    cubio_next = np.concatenate((cubio_pca, dc), axis=3)
    
    return cubio_next

def relu(cubio):
    cubio_relu = cubio * (cubio > 0)
    
    return cubio_relu
     
def one_stage_training(layer, n_comps, kernel, feature_list, X):
    print("training at the %dth layer ..." %(layer))
    patches_panel = flatten_patches(X, pow(2,layer))
    patches_mean_remov = remove_patches_mean(patches_panel)
    if layer == 1:
        patches_clean = remove_low_variance(patches_mean_remov)
    else:
        patches_clean = patches_mean_remov
    pca_kernel(patches_clean, n_comps[layer-1], k)
    cubio_next = relu(pca_transform(k, layer, patches_mean_remov, patches_panel, X_train))
    feature_list.append(cubio_next)
    print("Done! the shape of output cubio is %s." %(cubio_next.shape,))
    
def one_stage_training_v2(cubio, layer, n_comps, kernel, feature_list, X):
    print("training at the %dth layer ..." %(layer))
    patches_panel = flatten_patches_v2(cubio)
    patches_mean_remov = remove_patches_mean(patches_panel)
    if layer == 1:
        patches_clean = remove_low_variance(patches_mean_remov)
    else:
        patches_clean = patches_mean_remov
    pca_kernel(patches_clean,n_comps[layer-1], kernel)
    cubio_next = relu(pca_transform(kernel, layer, patches_mean_remov, patches_panel, X))
    feature_list.append(cubio_next)
    print("Done! the shape of output cubio is %s." %(cubio_next.shape,))
    
    return cubio_next

def feature_fusion(feature_list, num_layers):
    feature = feature_list[0].reshape(feature_list[0].shape[0], -1)
    for i in range(num_layers-1):
        feature = np.concatenate((feature,feature_list[i+1].reshape(feature_list[i+1].shape[0], -1)), axis=1)
    print("the shape of features we get is %s." %(feature.shape,))
    return feature

def Reduce_Feature(n_comps, feature):
    pca = PCA(n_components = n_comps)
    X_pc = pca.fit_transform(feature)
    print("the number of dimensions kept is %d." %(X_pc.shape[1]))
    
    return X_pc, pca

def F_test(percent, feature, label):
    Ftest = SelectPercentile(chi2, percent)
    X_f = Ftest.fit_transform(feature, label)
    print("the number of feature dimensions passing F-test is %d." %(X_f.shape[1]))

    return X_f, Ftest
    
def SVM_training(feature, label, n_comps, percent):
    print('SVM is under training...')
    X_f, Ftest = F_test(percent, feature, label)
    X_pc, pca = Reduce_Feature(n_comps, X_f)
    clf = SVC()
    clf.fit(X_pc, label)
    y_pred = clf.predict(X_pc)
    accuracy = accuracy_score(label, y_pred)
    print("SVM accuracy on training sample is %f" %(accuracy))
    
    return Ftest, pca, clf, accuracy

def SVM_training_v2(X_pc, label):
    print('SVM is under training...')
    clf = SVC()
    clf.fit(X_pc, label)
    y_pred = clf.predict(X_pc)
    accuracy = accuracy_score(label, y_pred)
    print("SVM accuracy on training sample is %f" %(accuracy))
    
    return clf, accuracy

def RF_training(feature, label, n_comps, percent):
    print('Random Forest is under training...')
    X_f, Ftest = F_test(percent, feature, label)
    X_pc, pca = Reduce_Feature(n_comps, X_f)
    clf = RandomForestClassifier()  
    clf.fit(X_pc, label)
    y_pred = clf.predict(X_pc)
    accuracy = accuracy_score(label, y_pred)
    print("RF accuracy on training sample is %f" %(accuracy))
    
    return Ftest, pca, clf, accuracy

def one_stage_testing(layer, kernel, feature_list, X):
    print("training at the %dth layer ..." %(layer))
    patches_panel = flatten_patches(X, pow(2,layer))
    patches_mean_remov = remove_patches_mean(patches_panel)
    cubio_next = relu(pca_transform(kernel, layer, patches_mean_remov, patches_panel, X))
    feature_list.append(cubio_next)
    print("Done! the shape of output cubio is %s." %(cubio_next.shape,))

def RF_testing(feature, label, Ftest, pca, clf):
    print("Test smaples are under Random Forest's testing...")
    feature_test = Ftest.transform(feature)
    print("the number of feature dimensions passing F-test is %d." %(feature_test.shape[1]))
    feature_pca = pca.transform(feature_test)
    print("the number of dimensions kept is %d." %(feature_pca.shape[1]))
    y_pred = clf.predict(feature_pca)
    accuracy = accuracy_score(label, y_pred)     
    print("RF accuracy on test sample is %f" %(accuracy))
    
    return accuracy, y_pred
          
def SVM_testing(feature, label, Ftest, pca, clf):
    print("Test smaples are under SVM's testing...")
    feature_test = Ftest.transform(feature)
    print("the number of feature dimensions passing F-test is %d." %(feature_test.shape[1]))
    feature_pca = pca.transform(feature_test)
    print("the number of dimensions kept is %d." %(feature_pca.shape[1]))
    y_pred = clf.predict(feature_pca)
    accuracy = accuracy_score(label, y_pred)     
    print("SVM accuracy on test sample is %f" %(accuracy))
    
    return accuracy, y_pred

# the first version -- parallel structure

## Training process

### Prepare the data

In [52]:
#prepare training parameters list
k = []
k_means = []

feature_list = []
n_comps = [3, 4, 7, 6, 8]

#preprocess input image batch and label batch
X_train, y_train_batch = image_set_preprocessing(X_train_raw, y_train_raw, batch_ratio = 1)
cubio = copy.deepcopy(X_train)

#compute the number of layers
layer_cnt = (int)(log(X_train.shape[1])/log(2))

### the first stage

In [53]:
cubio_1 = flatten_patches(cubio, 2)

cubio_1_remov = remove_patches_mean(cubio_1)

cubio_1_clean = remove_low_variance(cubio_1_remov,thr=0.05)

PCA_s = PCA(n_components=3).fit(cubio_1_clean)
k.append(PCA_s)
n_sample = X_train.shape[0]
h = 16
patches_proj = k[0].transform(cubio_1_remov)
cubio_pos = patches_proj.reshape(n_sample,h,h,-1)
cubio_neg = -cubio_pos
cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=3)
dc = cubio_1.mean(axis=1)*2
dc = dc.reshape(n_sample, h, h, -1)
cubio_next = relu(np.concatenate((cubio_pca, dc), axis=3))
feature_list.append(cubio_next)

### the second stage

In [54]:
cubio_2 = flatten_patches(cubio, 4)

cubio_2_remov = remove_patches_mean(cubio_2)

PCA_s = PCA(n_components=4)
k.append(PCA_s)

h = 8
patches_proj = k[1].fit_transform(cubio_2_remov)
cubio_pos = patches_proj.reshape(n_sample,h,h,-1)
cubio_neg = -cubio_pos
cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=3)
dc = cubio_2.mean(axis=1)*2
dc = dc.reshape(n_sample, h, h, -1)
cubio_next = relu(np.concatenate((cubio_pca, dc), axis=3))
feature_list.append(cubio_next)

### the third stage

In [58]:
cubio_3.shape

(960000, 64)

In [55]:
cubio_3 = flatten_patches(cubio, 8)

cubio_3_remov = remove_patches_mean(cubio_3)

PCA_s = MiniBatchSparsePCA(n_components=100, alpha=0.8, n_iter=20, batch_size=30, verbose=2)
k.append(PCA_s)

h = 4
patches_proj = k[2].fit_transform(cubio_3_remov)
cubio_pos = patches_proj.reshape(n_sample,h,h,-1)
cubio_neg = -cubio_pos
cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=3)
dc = cubio_3.mean(axis=1)*2
dc = dc.reshape(n_sample, h, h, -1)
cubio_next = relu(np.concatenate((cubio_pca, dc), axis=3))
feature_list.append(cubio_next)

Iteration   0 (elapsed time:  19s,  0.3mn)
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new random atom
Adding new ran

KeyboardInterrupt: 

### the fourth stage

In [208]:
cubio_4 = flatten_patches(cubio, 16)

cubio_4_remov = remove_patches_mean(cubio_4)

cubio_4_clean = cubio_4_remov

kmeans = KMeans(n_clusters=3, n_init=3, max_iter=50, verbose=0, random_state=0)
kmeans.fit(cubio_4_clean)
k_means.append(kmeans)


PCA_idx = kmeans.predict(cubio_4_remov)

PCA_obj = []
cubio_next = np.zeros((PCA_idx.shape[0], 37))
k.append(PCA_obj)
for cls in range(3):
    ind = PCA_idx==cls
    cubio_cls = cubio_4_remov[ind,:]
    PCA_obj.append(PCA(n_components = 6).fit(cubio_cls))
    n_sample = X_train.shape[0]
    h = 2
    cubio_pos = PCA_obj[cls].transform(cubio_cls)
    cubio_neg = -cubio_pos
    cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=1)
    cubio_next[ind,cls*12:(cls+1)*12] = cubio_pca
    dc = cubio_4[ind,:].mean(axis=1)*2
    cubio_next[ind,36] = dc

cubio_next = relu(cubio_next.reshape(X_train.shape[0], 2, 2, 37))
feature_list.append(cubio_next)

### the fifth stage

In [209]:
cubio_5 = flatten_patches(cubio, 32)

cubio_5_remov = remove_patches_mean(cubio_5)

cubio_5_clean = cubio_5_remov

kmeans = KMeans(n_clusters=3, n_init=3, max_iter=50, verbose=0, random_state=0)
kmeans.fit(cubio_5_clean)
k_means.append(kmeans)


PCA_idx = kmeans.predict(cubio_5_remov)

PCA_obj = []
cubio_next = np.zeros((PCA_idx.shape[0], 49))
k.append(PCA_obj)
for cls in range(3):
    ind = PCA_idx==cls
    cubio_cls = cubio_5_remov[ind,:]
    PCA_obj.append(PCA(n_components = 8).fit(cubio_cls))
    n_sample = X_train.shape[0]
    h = 1
    cubio_pos = PCA_obj[cls].transform(cubio_cls)
    cubio_neg = -cubio_pos
    cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=1)
    cubio_next[ind,cls*16:(cls+1)*16] = cubio_pca
    dc = cubio_5[ind,:].mean(axis=1)*2
    cubio_next[ind,48] = dc

cubio_next = relu(cubio_next.reshape(X_train.shape[0], 1, 1, 49))
feature_list.append(cubio_next)

### fuse the features

In [210]:
features = feature_fusion(feature_list, 5)

the shape of features we get is (60000, 3253).


### machine learning process

In [211]:
#feature selection
X_f, Ftest = F_test(50, features, y_train_batch)

#feature reduction
X_pc_32, pca_32 = Reduce_Feature(32, X_f)
X_pc_64, pca_64 = Reduce_Feature(64, X_f)
X_pc_128, pca_128 = Reduce_Feature(128, X_f)

#SVM training
svm_32, accuracy_svm_training_32= SVM_training_v2(X_pc_32, y_train_batch)
svm_64, accuracy_svm_training_64= SVM_training_v2(X_pc_64, y_train_batch)
svm_128, accuracy_svm_training_128= SVM_training_v2(X_pc_128, y_train_batch)

the number of feature dimensions passing F-test is 1626.
the number of dimensions kept is 32.
the number of dimensions kept is 64.
the number of dimensions kept is 128.
SVM is under training...
SVM accuracy on training sample is 0.995267
SVM is under training...
SVM accuracy on training sample is 0.994050
SVM is under training...
SVM accuracy on training sample is 0.990600


## Testing process

### prepare the data

In [212]:
#preprocess testing image batch and label batch 
X_test, y_test_batch = image_set_preprocessing(X_test_raw, y_test_raw, batch_ratio = 1)

#prepare testing parameters list
feature_list_test = []

### the first stage

In [213]:
cubio_1_test = flatten_patches(X_test, 2)

cubio_1_remov_test = remove_patches_mean(cubio_1_test)

n_sample = X_test.shape[0]
h = 16
patches_proj = k[0][0].transform(cubio_1_remov_test)
cubio_pos = patches_proj.reshape(n_sample,h,h,-1)
cubio_neg = -cubio_pos
cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=3)
dc = cubio_1_test.mean(axis=1)*2
dc = dc.reshape(n_sample, h, h, -1)
cubio_next = relu(np.concatenate((cubio_pca, dc), axis=3))
feature_list_test.append(cubio_next)

### the second stage

In [216]:
cubio_2_test = flatten_patches(X_test, 4)

cubio_2_remov_test = remove_patches_mean(cubio_2_test)

h = 8
patches_proj = k[1][0].transform(cubio_2_remov_test)
cubio_pos = patches_proj.reshape(n_sample,h,h,-1)
cubio_neg = -cubio_pos
cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=3)
dc = cubio_2_test.mean(axis=1)*2
dc = dc.reshape(n_sample, h, h, -1)
cubio_next = relu(np.concatenate((cubio_pca, dc), axis=3))
feature_list_test.append(cubio_next)

### the third stage

In [217]:
cubio_3_test = flatten_patches(X_test, 8)

cubio_3_remov_test = remove_patches_mean(cubio_3_test)

PCA_idx = k_means[0].predict(cubio_3_remov_test)

cubio_next = np.zeros((PCA_idx.shape[0], 43))

for cls in range(3):
    ind = PCA_idx==cls
    cubio_cls_test = cubio_3_remov_test[ind,:]
    h = 4
    cubio_pos = k[2][cls].transform(cubio_cls_test)
    cubio_neg = -cubio_pos
    cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=1)
    cubio_next[ind,cls*14:(cls+1)*14] = cubio_pca
    dc = cubio_3_test[ind,:].mean(axis=1)*2
    cubio_next[ind,42] = dc

cubio_next = relu(cubio_next.reshape(n_sample, 4, 4, 43))
feature_list_test.append(cubio_next)

### the fourth stage

In [218]:
cubio_4_test = flatten_patches(X_test, 16)

cubio_4_remov_test = remove_patches_mean(cubio_4_test)

PCA_idx = k_means[1].predict(cubio_4_remov_test)

cubio_next = np.zeros((PCA_idx.shape[0], 37))

for cls in range(3):
    ind = PCA_idx==cls
    cubio_cls = cubio_4_remov_test[ind,:]
    h = 2
    cubio_pos = k[3][cls].transform(cubio_cls)
    cubio_neg = -cubio_pos
    cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=1)
    cubio_next[ind,cls*12:(cls+1)*12] = cubio_pca
    dc = cubio_4_test[ind,:].mean(axis=1)*2
    cubio_next[ind,36] = dc

cubio_next = relu(cubio_next.reshape(n_sample, 2, 2, 37))
feature_list_test.append(cubio_next)

### the fifth stage

In [219]:
cubio_5_test = flatten_patches(X_test, 32)

cubio_5_remov_test = remove_patches_mean(cubio_5_test)

PCA_idx = k_means[2].predict(cubio_5_remov_test)

cubio_next = np.zeros((PCA_idx.shape[0], 49))

for cls in range(3):
    ind = PCA_idx==cls
    cubio_cls = cubio_5_remov_test[ind,:]
    h = 1
    cubio_pos = k[4][cls].transform(cubio_cls)
    cubio_neg = -cubio_pos
    cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=1)
    cubio_next[ind,cls*16:(cls+1)*16] = cubio_pca
    dc = cubio_5_test[ind,:].mean(axis=1)*2
    cubio_next[ind,48] = dc

cubio_next = relu(cubio_next.reshape(n_sample, 1, 1, 49))
feature_list_test.append(cubio_next)

### fuse  the features

In [220]:
features_test = feature_fusion(feature_list_test, 5)

the shape of features we get is (10000, 3253).


### classifying

In [221]:
#F-test --> PCA reducing dims --> SVM testing
accuracy_svm_testing_32, y_pred_svm_32 = SVM_testing(features_test, y_test_batch, Ftest, pca_32, svm_32)
accuracy_svm_testing_64, y_pred_svm_64 = SVM_testing(features_test, y_test_batch, Ftest, pca_64, svm_64)
accuracy_svm_testing_128, y_pred_svm_128 = SVM_testing(features_test, y_test_batch, Ftest, pca_128, svm_128)

Test smaples are under SVM's testing...
the number of feature dimensions passing F-test is 1626.
the number of dimensions kept is 32.
SVM accuracy on test sample is 0.979900
Test smaples are under SVM's testing...
the number of feature dimensions passing F-test is 1626.
the number of dimensions kept is 64.
SVM accuracy on test sample is 0.981800
Test smaples are under SVM's testing...
the number of feature dimensions passing F-test is 1626.
the number of dimensions kept is 128.
SVM accuracy on test sample is 0.981300


# The second version -- tree structure

## Training process

### prepare the data

In [243]:
#prepare training parameters list
k = []
k_means = []

feature_list = []
n_comps = [3, 4, 7, 6, 8]

#preprocess input image batch and label batch
X_train, y_train_batch = image_set_preprocessing(X_train_raw, y_train_raw, batch_ratio = 1)
cubio = copy.deepcopy(X_train)

#compute the number of layers
layer_cnt = (int)(log(X_train.shape[1])/log(2))

### the first stage

In [244]:
cubio_1 = flatten_patches_v2(cubio)

cubio_1_remov = remove_patches_mean(cubio_1)

cubio_1_clean = remove_low_variance(cubio_1_remov,thr=0.05)

PCA_obj = []
k.append(PCA_obj)

PCA_obj.append(PCA(n_components = 3).fit(cubio_1_clean))
n_sample = X_train.shape[0]
h = 16
patches_proj = PCA_obj[0].transform(cubio_1_remov)
cubio_pos = patches_proj.reshape(n_sample,h,h,-1)
cubio_neg = -cubio_pos
cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=3)
dc = cubio_1.mean(axis=1)*2
dc = dc.reshape(n_sample, h, h, -1)
cubio_next = relu(np.concatenate((cubio_pca, dc), axis=3))
feature_list.append(cubio_next)

### the second stage

In [245]:
cubio_2 = flatten_patches_v2(cubio_next)

cubio_2_remov = remove_patches_mean(cubio_2)

cubio_2_clean = cubio_2_remov

PCA_obj = []
k.append(PCA_obj)

PCA_obj.append(PCA(n_components = 4).fit(cubio_2_clean))
h = 8
patches_proj = PCA_obj[0].transform(cubio_2_remov)
cubio_pos = patches_proj.reshape(n_sample,h,h,-1)
cubio_neg = -cubio_pos
cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=3)
dc = cubio_2.mean(axis=1)*2
dc = dc.reshape(n_sample, h, h, -1)
cubio_next = relu(np.concatenate((cubio_pca, dc), axis=3))
feature_list.append(cubio_next)

In [246]:
cubio_next.shape

(60000, 8, 8, 9)

### the third stage

In [247]:
cubio_3 = flatten_patches_v2(cubio_next)

cubio_3_remov = remove_patches_mean(cubio_3)

cubio_3_clean = cubio_3_remov

kmeans = KMeans(n_clusters=3, n_init=3, max_iter=50, verbose=0, random_state=0)
kmeans.fit(cubio_3_clean)
k_means.append(kmeans)

PCA_idx = kmeans.predict(cubio_3_remov)

PCA_obj = []
cubio_next = np.zeros((PCA_idx.shape[0], 43))
k.append(PCA_obj)
for cls in range(3):
    ind = PCA_idx==cls
    cubio_cls = cubio_3_remov[ind,:]
    PCA_obj.append(PCA(n_components = 7).fit(cubio_cls))
    n_sample = X_train.shape[0]
    h = 4
    cubio_pos = PCA_obj[cls].transform(cubio_cls)
    cubio_neg = -cubio_pos
    cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=1)
    cubio_next[ind,cls*14:(cls+1)*14] = cubio_pca
    dc = cubio_3[ind,:].mean(axis=1)*2
    cubio_next[ind,42] = dc

cubio_next = relu(cubio_next.reshape(X_train.shape[0], 4, 4, 43))
feature_list.append(cubio_next)

### the fourth stage

In [248]:
cubio_4 = flatten_patches_v2(cubio_next)

cubio_4_remov = remove_patches_mean(cubio_4)

cubio_4_clean = cubio_4_remov

kmeans = KMeans(n_clusters=3, n_init=3, max_iter=50, verbose=0, random_state=0)
kmeans.fit(cubio_4_clean)
k_means.append(kmeans)


PCA_idx = kmeans.predict(cubio_4_remov)

PCA_obj = []
cubio_next = np.zeros((PCA_idx.shape[0], 37))
k.append(PCA_obj)
for cls in range(3):
    ind = PCA_idx==cls
    cubio_cls = cubio_4_remov[ind,:]
    PCA_obj.append(PCA(n_components = 6).fit(cubio_cls))
    n_sample = X_train.shape[0]
    h = 2
    cubio_pos = PCA_obj[cls].transform(cubio_cls)
    cubio_neg = -cubio_pos
    cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=1)
    cubio_next[ind,cls*12:(cls+1)*12] = cubio_pca
    dc = cubio_4[ind,:].mean(axis=1)*2
    cubio_next[ind,36] = dc

cubio_next = relu(cubio_next.reshape(X_train.shape[0], 2, 2, 37))
feature_list.append(cubio_next)

### the fifth stage

In [249]:
cubio_5 = flatten_patches_v2(cubio_next)

cubio_5_remov = remove_patches_mean(cubio_5)

cubio_5_clean = cubio_5_remov

kmeans = KMeans(n_clusters=3, n_init=3, max_iter=50, verbose=0, random_state=0)
kmeans.fit(cubio_5_clean)
k_means.append(kmeans)


PCA_idx = kmeans.predict(cubio_5_remov)

PCA_obj = []
cubio_next = np.zeros((PCA_idx.shape[0], 49))
k.append(PCA_obj)
for cls in range(3):
    ind = PCA_idx==cls
    cubio_cls = cubio_5_remov[ind,:]
    PCA_obj.append(PCA(n_components = 8).fit(cubio_cls))
    n_sample = X_train.shape[0]
    h = 1
    cubio_pos = PCA_obj[cls].transform(cubio_cls)
    cubio_neg = -cubio_pos
    cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=1)
    cubio_next[ind,cls*16:(cls+1)*16] = cubio_pca
    dc = cubio_5[ind,:].mean(axis=1)*2
    cubio_next[ind,48] = dc

cubio_next = relu(cubio_next.reshape(X_train.shape[0], 1, 1, 49))
feature_list.append(cubio_next)

### fuse the features

In [250]:
features = feature_fusion(feature_list, 5)

the shape of features we get is (60000, 3253).


### machine learning process

In [251]:
#feature selection
X_f, Ftest = F_test(50, features, y_train_batch)

#feature reduction
X_pc_32, pca_32 = Reduce_Feature(32, X_f)
X_pc_64, pca_64 = Reduce_Feature(64, X_f)
X_pc_128, pca_128 = Reduce_Feature(128, X_f)

#SVM training
svm_32, accuracy_svm_training_32= SVM_training_v2(X_pc_32, y_train_batch)
svm_64, accuracy_svm_training_64= SVM_training_v2(X_pc_64, y_train_batch)
svm_128, accuracy_svm_training_128= SVM_training_v2(X_pc_128, y_train_batch)

the number of feature dimensions passing F-test is 1626.
the number of dimensions kept is 32.
the number of dimensions kept is 64.
the number of dimensions kept is 128.
SVM is under training...
SVM accuracy on training sample is 0.994650
SVM is under training...
SVM accuracy on training sample is 0.993083
SVM is under training...
SVM accuracy on training sample is 0.988700


## Testing process

### prepare the data

In [252]:
#preprocess testing image batch and label batch 
X_test, y_test_batch = image_set_preprocessing(X_test_raw, y_test_raw, batch_ratio = 1)
cubio = copy.deepcopy(X_test)

#prepare testing parameters list
feature_list_test = []

### the first stage

In [253]:
cubio_1_test = flatten_patches_v2(cubio)

cubio_1_remov_test = remove_patches_mean(cubio_1_test)

n_sample = X_test.shape[0]
h = 16
patches_proj = k[0][0].transform(cubio_1_remov_test)
cubio_pos = patches_proj.reshape(n_sample,h,h,-1)
cubio_neg = -cubio_pos
cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=3)
dc = cubio_1_test.mean(axis=1)*2
dc = dc.reshape(n_sample, h, h, -1)
cubio_next = relu(np.concatenate((cubio_pca, dc), axis=3))
feature_list_test.append(cubio_next)

### the second stage

In [254]:
cubio_2_test = flatten_patches_v2(cubio_next)

cubio_2_remov_test = remove_patches_mean(cubio_2_test)

h = 8
patches_proj = k[1][0].transform(cubio_2_remov_test)
cubio_pos = patches_proj.reshape(n_sample,h,h,-1)
cubio_neg = -cubio_pos
cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=3)
dc = cubio_2_test.mean(axis=1)*2
dc = dc.reshape(n_sample, h, h, -1)
cubio_next = relu(np.concatenate((cubio_pca, dc), axis=3))
feature_list_test.append(cubio_next)

### the third stage

In [255]:
cubio_3_test = flatten_patches_v2(cubio_next)

cubio_3_remov_test = remove_patches_mean(cubio_3_test)

PCA_idx = k_means[0].predict(cubio_3_remov_test)

cubio_next = np.zeros((PCA_idx.shape[0], 43))

for cls in range(3):
    ind = PCA_idx==cls
    cubio_cls_test = cubio_3_remov_test[ind,:]
    h = 4
    cubio_pos = k[2][cls].transform(cubio_cls_test)
    cubio_neg = -cubio_pos
    cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=1)
    cubio_next[ind,cls*14:(cls+1)*14] = cubio_pca
    dc = cubio_3_test[ind,:].mean(axis=1)*2
    cubio_next[ind,42] = dc

cubio_next = relu(cubio_next.reshape(n_sample, 4, 4, 43))
feature_list_test.append(cubio_next)

### the fourth stage

In [259]:
cubio_4_test = flatten_patches_v2(cubio_next)

cubio_4_remov_test = remove_patches_mean(cubio_4_test)

PCA_idx = k_means[1].predict(cubio_4_remov_test)

cubio_next = np.zeros((PCA_idx.shape[0], 37))

for cls in range(3):
    ind = PCA_idx==cls
    cubio_cls = cubio_4_remov_test[ind,:]
    h = 2
    cubio_pos = k[3][cls].transform(cubio_cls)
    cubio_neg = -cubio_pos
    cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=1)
    cubio_next[ind,cls*12:(cls+1)*12] = cubio_pca
    dc = cubio_4_test[ind,:].mean(axis=1)*2
    cubio_next[ind,36] = dc

cubio_next = relu(cubio_next.reshape(n_sample, 2, 2, 37))
feature_list_test.append(cubio_next)

### the fifth stage

In [260]:
cubio_5_test = flatten_patches_v2(cubio_next)

cubio_5_remov_test = remove_patches_mean(cubio_5_test)

PCA_idx = k_means[2].predict(cubio_5_remov_test)

cubio_next = np.zeros((PCA_idx.shape[0], 49))

for cls in range(3):
    ind = PCA_idx==cls
    cubio_cls = cubio_5_remov_test[ind,:]
    h = 1
    cubio_pos = k[4][cls].transform(cubio_cls)
    cubio_neg = -cubio_pos
    cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=1)
    cubio_next[ind,cls*16:(cls+1)*16] = cubio_pca
    dc = cubio_5_test[ind,:].mean(axis=1)*2
    cubio_next[ind,48] = dc

cubio_next = relu(cubio_next.reshape(n_sample, 1, 1, 49))
feature_list_test.append(cubio_next)

### fuse the features

In [261]:
features_test = feature_fusion(feature_list_test, 5)

the shape of features we get is (10000, 3253).


### classifying

In [263]:
#F-test --> PCA reducing dims --> SVM testing
accuracy_svm_testing_32, y_pred_svm_32 = SVM_testing(features_test, y_test_batch, Ftest, pca_32, svm_32)
accuracy_svm_testing_64, y_pred_svm_64 = SVM_testing(features_test, y_test_batch, Ftest, pca_64, svm_64)
accuracy_svm_testing_128, y_pred_svm_128 = SVM_testing(features_test, y_test_batch, Ftest, pca_128, svm_128)

Test smaples are under SVM's testing...
the number of feature dimensions passing F-test is 1626.
the number of dimensions kept is 32.
SVM accuracy on test sample is 0.980800
Test smaples are under SVM's testing...
the number of feature dimensions passing F-test is 1626.
the number of dimensions kept is 64.
SVM accuracy on test sample is 0.981600
Test smaples are under SVM's testing...
the number of feature dimensions passing F-test is 1626.
the number of dimensions kept is 128.
SVM accuracy on test sample is 0.978900


In [3]:
import logging
from time import time

from numpy.random import RandomState
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_olivetti_faces
from sklearn.cluster import MiniBatchKMeans
from sklearn import decomposition

image_shape = (64, 64)
rng = RandomState(0)

In [4]:
# Load faces data
dataset = fetch_olivetti_faces(shuffle=True, random_state=rng)
faces = dataset.data

n_samples, n_features = faces.shape

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /Users/shanlinsun/scikit_learn_data


In [15]:
# global centering
faces_centered = faces - faces.mean(axis=0)

In [18]:
# local centering
faces_centered -= faces_centered.mean(axis=1).reshape(n_samples, -1)

In [17]:
faces_centered.mean(axis=1).reshape(n_samples, -1).shape

(400, 1)

In [20]:
estimator = decomposition.MiniBatchSparsePCA(n_components=6, alpha=0.8,
                                      n_iter=100, batch_size=3,
                                      random_state=rng)

In [21]:
estimator.fit(faces_centered)

MiniBatchSparsePCA(alpha=0.8, batch_size=3, callback=None, method='lars',
          n_components=6, n_iter=100, n_jobs=1,
          random_state=<mtrand.RandomState object at 0x11cb0dd38>,
          ridge_alpha=0.01, shuffle=True, verbose=False)

In [22]:
components_ = estimator.components_
components_.shape

(6, 4096)

In [25]:
components_[:6].shape

(6, 4096)